![Java 虚拟机加载类](Java 虚拟机加载类.png)

## 概述
- 三大步骤：**加载**、**链接**、**初始化**
  - 链接过程中同样需要验证
  - 内存中的类没有经过初始化，同样不能使用

### Java 语言分类
- 大分类
  - 基础类型
  - 引用类型
    - 类
    - 接口
    - 数组类
    - 泛型参数(编译过程中会被抹去)

- java 虚拟机只有前三种类型
  - 数组类：由 java 虚拟机直接生成
  - 类、接口：有对应的字节流

---

## 加载
- **是指查找字节流，并且据此创建类的过程**
- 借助**类加载器**来完成查找字节流的过程

### 类加载器
- 启动类加载器（bootstrap class loader）
  - 由 C++ 实现，没有对应的 Java 对象，在 Java 中只能用 null 来指代
- 其他类加载器都是 java.lang.ClassLoader 的子类
  - 需要先由另一个加载器（比如启动类加载器）加载至虚拟机中，方能执行类加载

#### 双亲委派模型
- 每当一个类加载器接收到加载请求时，它会先将请求转发给父类加载器。
- 在父类加载器没有找到所请求的类的情况下，该类加载器才会尝试去加载。

#### 版本区分
- Java 9 之前
  - **启动类加载器**
    - 负责加载最为基础、最为重要的类，
    - 比如存放在 JRE 的 lib 目录下 jar 包中的类
    - 由虚拟机参数 -Xbootclasspath 指定的类
  - **扩展类加载器**（extension class loader），由 Java 核心类库提供
    - 父类是启动类加载器
    - 负责加载相对次要、但又通用的类
    - 比如存放在 JRE 的 lib/ext 目录下 jar 包中的类
    - 由系统变量 java.ext.dirs 指定的类
  - **应用类加载器**（application class loader），由 Java 核心类库提供
    - 父类是扩展类加载器
    - 负责加载应用程序路径下的类
    - 应用程序路径，指虚拟机参数 -cp/-classpath、系统变量 java.class.path 或环境变量 CLASSPATH 所指定的路径
    - 默认情况下，应用程序中包含的类便是由应用类加载器加载的

- Java 9 及之后
  - 引入了模块系统，并且略微更改了上述的类加载器
  - **扩展类加载器**：被改名为**平台类加载器**（platform class loader）
  - Java SE 中除了少数几个关键模块，比如说 java.base 是由启动类加载器加载之外，其他的模块均由平台类加载器所加载

#### 自定义类加载器
- 可以加入自定义的类加载器，来实现特殊的加载方式
- 举例来说，可以对 class 文件进行加密，加载时再利用自定义的类加载器对其解密

#### 命名空间
- **类加载器还提供了命名空间的作用**
- 在 Java 虚拟机中，类的唯一性是由**类加载器实例**以及**类的全名**一同确定的
- 在大型应用中，往往借助这一特性，来运行同一个类的不同版本

---

## 链接
- 指将创建成的类合并至 Java 虚拟机中，使之能够执行的过程
- 它可分为**验证**、**准备**以及**解析**三个阶段

### 验证
- 目的：确保被加载类能够满足 Java 虚拟机的约束条件
- 通常而言，Java 编译器生成的类文件必然满足 Java 虚拟机的约束条件
  - 讲解字节码注入时再详细介绍

### 准备
- 目的1：被加载类的静态字段分配内存
  - Java 代码中对静态字段的具体初始化，则会在稍后的初始化阶段中进行
- 目的2：部分 Java 虚拟机还会在此阶段构造其他跟类层次相关的数据结构，比如说用来实现虚方法的动态绑定的方法表
  - 在 class 文件被加载至 Java 虚拟机之前，这个类无法知道其他类及其方法、字段所对应的具体地址，甚至不知道自己方法、字段的地址
  - 当需要引用这些成员时，Java 编译器会生成一个符号引用。在运行阶段，这个符号引用一般都能够无歧义地定位到具体目标上
    - 比如：一个方法调用，编译器会生成一个包含目标方法所在类的名字、目标方法的名字、接收参数类型以及返回值类型的符号引用，来指代所要调用的方法

### 解析
- 将上面生成的**符号引用**解析成为**实际引用**
  - 如果符号引用指向一个未被加载的类，或者未被加载类的字段或方法，那么解析将触发这个类的加载（但未必触发这个类的链接以及初始化）
- Java 虚拟机规范并没有要求在链接过程中完成解析
  - 它仅规定了：**如果某些字节码使用了符号引用，那么在执行这些字节码之前，需要完成对这些符号引用的解析**

---

## 初始化
- Java 中，初始化一个静态字段方法
  - 在声明时直接赋值
  - 静态代码块中对其赋值
- 直接赋值的静态字段如果被 final 修饰，且类型是基本类型或字符串类型，那么改字段会被标记为常量值，其初始化由 Java 虚拟机完成
- 除此之外的直接赋值操作，包括静态代码块中的代码，会被 Java 编译器置于同一个方法中，并命名为 < clinit >
  - Java 虚拟机会通过加锁来确保类的 < clinit > 方法仅被执行一次
- 只有当初始化完成之后，类才正式成为可执行的状态
- 类的初始化何时会被触发?
  1. 当虚拟机启动时，初始化用户指定的主类
  2. 当遇到用以新建目标类实例的 new 指令时，初始化 new 指令的目标类
  3. 当遇到调用静态方法的指令时，初始化该静态方法所在的类
  4. 当遇到访问静态字段的指令时，初始化该静态字段所在的类
  5. 子类的初始化会触发父类的初始化
  6. 如果一个接口定义了 default 方法，那么直接实现或者间接实现该接口的类的初始化，会触发该接口的初始化
  7. 使用反射 API 对某个类进行反射调用时，初始化这个类
  8. 当初次调用 MethodHandle 实例时，初始化该 MethodHandle 指向的方法所在的类

---

### 著名案例
- **单例延迟初始化例子**
- 只有当调用 Singleton.getInstance 时，程序才会访问 LazyHolder.INSTANCE，才会触发对 LazyHolder 的初始化（对应上面第 4 种情况）
- 由于类初始化是线程安全的，并且仅被执行一次，因此程序可以确保多线程环境下有且仅有一个 Singleton 实例。

```
public class Singleton {
  private Singleton() {}
  private static class LazyHolder {
    static final Singleton INSTANCE = new Singleton();
  }
  public static Singleton getInstance() {
    return LazyHolder.INSTANCE;
  }
}
```

---

## 课后练习
### 练习1
- 通过 JVM 参数 -verbose:class 来打印类加载的先后顺序，并且在 LazyHolder 的初始化方法中打印特定字样
- 在命令行中运行下述指令（不包含提示符 &amp;）

``` shell
$ echo '
public class Singleton {
  private Singleton() {}
  private static class LazyHolder {
    static final Singleton INSTANCE = new Singleton();
    static {
      System.out.println("LazyHolder.<clinit>");
    }
  }
  public static Object getInstance(boolean flag) {
    if (flag) return new LazyHolder[2];  // 新建数组
    return LazyHolder.INSTANCE;
  }
  public static void main(String[] args) {
    getInstance(true);
    System.out.println("----");
    getInstance(false);
  }
}' > Singleton.java
$ javac Singleton.java
$ java -verbose:class Singleton
```

- 问题 1：新建数组会导致 LazyHolder 的加载吗？会导致它的初始化吗？
  - 新建数组会导致 LazyHolder 的加载
  - 不会初始化，因为这个类并没有使用到（没有达到初始化的条件）

``` log
## log 信息
[Loaded Singleton from file:/Users/henry/Downloads/]             // Singleton 类加载
其他：...
[Loaded Singleton.LazyHolder from file:/Users/henry/Downloads/]  // LazyHolder 类加载（新建数组，即 getInstance(true)）
----                                                             // System.out
LazyHolder.<clinit>                                              // Singleton.LazyHolder 初始化（即 getInstance(false)）
```

### 练习2
- 在命令行中运行下述指令（不包含提示符 &amp;）

``` shell
$ java -cp /path/to/asmtools.jar org.openjdk.asmtools.jdis.Main Singleton\$LazyHolder.class > Singleton\$LazyHolder.jasm.1
$ awk 'NR==1,/stack 1/{sub(/stack 1/, "stack 0")} 1' Singleton\$LazyHolder.jasm.1 > Singleton\$LazyHolder.jasm
$ java -cp /path/to/asmtools.jar org.openjdk.asmtools.jasm.Main Singleton\$LazyHolder.jasm
$ java -verbose:class Singleton
```

- 问题 2：新建数组会导致 LazyHolder 的链接吗？
  - 新建数组的时候并不是要使用这个类（只是定义了放这个类的容器）所以不会被链接
  - 调用 getInstance(false) 的时候，约等于告诉虚拟机，要使用这个类了，把这个类造好（链接），然后把 static 修饰的字符赋予变量（初始化）

---